In [1]:
using Statistics, BenchmarkTools, LinearAlgebra, SparseArrays, CUDA
using CUDA.CUSPARSE

- Función para generar la matriz A y el vector b de dimensión $n \times n$ y $n$, respectivamente cuya solución será un vector de 1.

In [2]:
function get_A_b(N)
    dv = fill(3., (N,))
    ev = fill(-1., (N-1,))
    A = Tridiagonal(ev,dv, ev)
    A = sparse(A)
    b = fill(1.5, (N,))
    b[1] = b[end] = 5/2
    isodd(N) ? b[ceil(Int, N/2)] = 1 : b[trunc(Int, N/2)] = b[trunc(Int, N/2+1)] = 1
    for row ∈ 1:ceil(Int,N/2)
        if isodd(N)
            if row != ceil(N/2)
                A[row,end-row+1] = A[end-row+1, row] = 0.5
            end
        else
            if (row != N/2) & (row != (N/2 + 1))
                A[row,end-row+1] = A[end-row+1, row] = 0.5
            end
        end
    end
    return A, b
end

get_A_b (generic function with 1 method)

## Unpreconditioned version of the Conjugate Gradient Algorithm in Julia


In [3]:
# Algoritmo sacado directo del pseudocódigo
function CG(A,b,x,r,ρ,res,p,ϵ)
    i = 1
    while  norm(res) > ϵ
        # Aquí iría la condición más chida de precondicionamiento
        ρ[1] = ρ[2] # Utilizamos sólo dos actualizaciones hacia atrás, por eso esto sí es una lista.
        ρ[2] = r'*r # Aquí cambia cuando es precondicionada
        if i == 1
            p = r
        else
            β = ρ[2]/ρ[1]
            p = r + β*p
        end
        q = A*p
        α = ρ[2]/(p'*q)
        x = x + α*p
        r = r - α*q
        i += 1
        res = b - A*x
    end
    return x
end

CG (generic function with 1 method)

### Secuencial

In [4]:
n = 25_000
A, b = get_A_b(n)
x = rand(n)
r = b - A*x
ρ = [0, r'*r]
res = b - A*x
p = Array{Float64}(undef, size(r,1), 1);
ϵ = 0.0001
@btime CG(A,b,x,r,ρ,res,p,ϵ)

  7.288 ms (635 allocations: 37.40 MiB)


25000-element Array{Float64,1}:
 0.9999998741141164
 0.9999996766562045
 0.9999998570370346
 1.0000001714175357
 1.0000003697871556
 1.0000005528021825
 1.0000003963116992
 1.0000002511014212
 1.000000086241815
 1.0000000233947726
 0.9999996059764295
 0.9999995796451858
 0.9999994559611934
 ⋮
 1.0000002313434646
 0.9999999133049821
 0.9999998506862864
 0.9999999127616727
 0.9999996594687439
 0.9999995877626957
 0.9999997723899025
 0.9999999178946124
 1.0000001168967796
 1.0000002394855256
 1.0000004587325615
 1.0000001900042221

### Paralelo

In [5]:
dA = CuSparseMatrixCSR(A)
db = CuArray(b)
dx = CuArray(x)
dr = db - dA*dx
d = dr'*dr
ρ = [0.0, d]
dres = db - dA*dx
ϵ = 0.0001
dp = CuArray{Float64}(undef, size(dr,1), 1);
@btime CG(dA,db,dx,dr,ρ,dres,dp,ϵ)

  3.106 ms (6228 allocations: 140.44 KiB)


25000-element CuArray{Float64,1}:
 0.9999998741141164
 0.9999996766562047
 0.9999998570370349
 1.000000171417536
 1.0000003697871558
 1.0000005528021827
 1.0000003963116992
 1.0000002511014214
 1.000000086241815
 1.0000000233947723
 0.9999996059764292
 0.9999995796451856
 0.9999994559611932
 ⋮
 1.0000002313434646
 0.9999999133049823
 0.9999998506862868
 0.9999999127616729
 0.9999996594687439
 0.9999995877626955
 0.9999997723899025
 0.9999999178946124
 1.0000001168967796
 1.0000002394855256
 1.0000004587325615
 1.000000190004222

### Paralelo especificando tipo de dato

In [7]:
dA = convert(SparseMatrixCSC{Float32, Int32}, A)
dA = CuSparseMatrixCSR(dA)
db = CuArray{Float32}(b)
dx = CuArray{Float32}(x)
dr = db - dA*dx
d = dr'*dr
ρ = [0.0f0, d]
dres = db - dA*dx
ϵ = 0.0001f0
dp = CuArray{Float32}(undef, size(dr,1), 1);
@btime CG(dA,db,dx,dr,ρ,dres,dp,ϵ)

  1.892 ms (6513 allocations: 146.88 KiB)


25000-element CuArray{Float32,1}:
 0.99999976
 1.0000002
 1.0000004
 1.0000004
 1.0
 1.0000001
 1.0
 0.9999998
 1.0000001
 0.99999976
 0.99999964
 0.9999995
 0.99999964
 ⋮
 0.9999998
 1.0
 1.0000002
 1.0000001
 0.99999994
 1.0000001
 0.9999999
 1.0000001
 0.99999994
 0.9999997
 0.9999998
 0.9999998